In [1]:
import numpy as np
from ipywidgets import widgets
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0}) # garbage collection should close plots when finished so safe to supress this warning
%matplotlib widget

In [136]:
def get_hd(fs_rf, interf_f):
    hd2 = [fs_rf - (2*i % fs_rf) if 2*i % fs_rf >= fs_rf/2 else (2*i % fs_rf) for i in interf_f]
    hd3 = [fs_rf - (3*i % fs_rf) if 3*i % fs_rf >= fs_rf/2 else (3*i % fs_rf) for i in interf_f]
    hd4 = [fs_rf - (4*i % fs_rf) if 4*i % fs_rf >= fs_rf/2 else (4*i % fs_rf) for i in interf_f]
    hd5 = [fs_rf - (5*i % fs_rf) if 5*i % fs_rf >= fs_rf/2 else (5*i % fs_rf) for i in interf_f]
    return hd2, hd3, hd4, hd5

def get_il(fs_rf, signal_f, il_factor):
    if il_factor > 1:
        il1 = [abs(fs_rf/2 - i) for i in signal_f]
    else:
        il1 = 0
        
    if il_factor > 2:
        il2 = [abs(fs_rf/4 - i) for i in signal_f]
        il3 = [abs(fs_rf/4 + i) if ((fs_rf/4 + i) < fs_rf/2) else abs(fs_rf/4*3-i) for i in signal_f]
    else:
        il2 = il3 = 0
        
    if il_factor > 4:
        il4 = [abs(fs_rf/8 - i) for i in signal_f]
        il5 = [abs(fs_rf/8 + i) if ((fs_rf/8 + i) < fs_rf/2) else abs(fs_rf/8*7-i) for i in signal_f]
        il6 = [abs(fs_rf/8*3-i) for i in signal_f]
        il7 = [abs(fs_rf/8*3+i) if ((fs_rf/8*3+i) < fs_rf/2) else abs(fs_rf/8*5-i) for i in signal_f]
    else:
        il4 = il5 = il6 = il7 = 0
        
    return il1, il2, il3, il4, il5, il6, il7

def get_fs_hd(il_factor, fs_rf, hd):
    hd2, hd3, hd4, hd5 = hd
    
    if il_factor > 1:
        fs2_m_hd2 = [abs(fs_rf/2-i) for i in hd2]
        fs2_m_hd3 = [fs_rf/2-i for i in hd3]
    else:
        fs2_m_hd2 = fs2_m_hd3 = 0
    
    if il_factor > 2:
        fs4_m_hd2 = [abs(fs_rf/4-i) for i in hd2]
        fs4_p_hd2 = [abs(fs_rf/4+i) if (fs_rf/4+i < fs_rf/2) else abs(fs_rf/4*3-i) for i in hd2]
        fs4_m_hd3 = [abs(fs_rf/4-i) for i in hd3]
        fs4_p_hd3 = [abs(fs_rf/4+i) if (fs_rf/4+i < fs_rf/2) else abs(fs_rf/4*3-i) for i in hd3]    
    else:
        fs4_m_hd2 = fs4_p_hd2 = fs4_m_hd3 = fs4_p_hd3 = 0
        
    if il_factor > 4:
        fs8_m_hd2 = [abs(fs_rf/8-i) for i in hd2]
        fs8_p_hd2 = [abs(fs_rf/8+i) if (fs_rf/8+i < fs_rf/2) else abs(fs_rf/8*7-i) for i in hd2]
        fs8_m_hd3 = [abs(fs_rf/8-i) for i in hd3]
        fs8_p_hd3 = [abs(fs_rf/8+i) if (fs_rf/8+i < fs_rf/2) else abs(fs_rf/8*7-i) for i in hd3]
    else:
        fs8_m_hd2 = fs8_p_hd2 = fs8_m_hd3 = fs8_p_hd3 = 0
        
    return fs2_m_hd2, fs2_m_hd3, fs4_m_hd2, fs4_p_hd2, fs4_m_hd3, fs4_p_hd3, fs8_m_hd2, fs8_p_hd2, fs8_m_hd3, fs8_p_hd3

def get_signals(fs_rf, fc, fs_bw, pll_ref, il_factor):
    f = np.linspace(-0.5,0.5,100)
    signal_f = [fs_rf - ((fc+fs_bw*i) % fs_rf) if ((fc+fs_bw*i) % fs_rf) >= fs_rf/2 else ((fc+fs_bw*i) % fs_rf) for i in f]
    interf_f = [fc+(fs_bw*i) for i in f]
    hd = get_hd(fs_rf, interf_f)
    il = get_il(fs_rf, signal_f, il_factor)
    fs_hd = get_fs_hd(il_factor, fs_rf, hd)
    clk_mix_prod_up = [fs_rf - ((fc+fs_bw*i+pll_ref) % fs_rf) if ((fc+fs_bw*i+pll_ref) % fs_rf) >= fs_rf/2 else ((fc+fs_bw*i+pll_ref) % fs_rf) for i in f]
    clk_mix_prod_down = [fs_rf - ((fc+fs_bw*i-pll_ref) % fs_rf) if ((fc+fs_bw*i-pll_ref) % fs_rf) >= fs_rf/2 else ((fc+fs_bw*i-pll_ref) % fs_rf) for i in f]
    
    return f, signal_f, interf_f, hd, il, fs_hd, clk_mix_prod_up, clk_mix_prod_down

    
def make_plot(signal_f, fs_rf, hd, il, fs_hd):
    
    # Get HD
    hd2, hd3, hd4, hd5 = hd
    
    # get IL
    il1, il2, il3, il4, il5, il6, il7 = il
    
    # get FS/N +- HD
    fs2_m_hd2, fs2_m_hd3, fs4_m_hd2, fs4_p_hd2, fs4_m_hd3, fs4_p_hd3, fs8_m_hd2, fs8_p_hd2, fs8_m_hd3, fs8_p_hd3 = fs_hd
    
    fig, ax = plt.subplots(figsize=(9,5))
    ax.grid(True)
    ax.set_xlabel('Frequency (MHz)', fontsize=10)
    ax.set_ylabel('Harmonic No.', fontsize=10)
    ax.set_xlim([0, 4096/2 + 4096*0.01])
    ax.set_ylim([0, 7])
    ax.grid(True)
    
    # Rx band
    rx_line_a, = ax.plot([min(signal_f), max(signal_f)], np.repeat(6,2), label='Rx Band', marker='o', color='tab:blue')
    rx_line_b = ax.axvline(x=min(signal_f), ymin=0, ymax=6/7, color='tab:blue')
    rx_line_c = ax.axvline(x=max(signal_f), ymin=0, ymax=6/7, color='tab:blue')
    
    # Nyquist
    nyq_line_a, = ax.plot(fs_rf/2, 6, marker='o', color='tab:gray', label='Nyquist')
    nyq_line_b  = ax.axvline(x=fs_rf/2, ymin=0, ymax=6/7, color='tab:gray')
    
    # Harmonics
    hd2_line, = ax.plot([min(hd2),max(hd2)], np.repeat(2,2), label='HD2', marker='o', color='tab:orange')
    hd3_line, = ax.plot([min(hd3),max(hd3)], np.repeat(3,2), label='HD3', marker='o', color='tab:green')
    hd4_line, = ax.plot([min(hd4),max(hd4)], np.repeat(4,2), label='HD4', marker='o', color='tab:red')
    hd5_line, = ax.plot([min(hd5),max(hd5)], np.repeat(5,2), label='HD5', marker='o', color='tab:olive')
    
    # Interleaving Spur
    il1_line, = ax.plot([min(il1),max(il1)], np.repeat(0.4,2), label='IL RX1', marker='o', color='tab:purple')
    il2_line, = ax.plot([min(il2),max(il2)], np.repeat(0.5,2), label='IL RX2', marker='o', color='tab:brown')
    il3_line, = ax.plot([min(il3),max(il3)], np.repeat(0.6,2), label='IL RX3', marker='o', color='tab:pink')
    il4_line, = ax.plot([min(il4),max(il4)], np.repeat(0.7,2), label='IL RX4', marker='o', color='tab:gray')
    il5_line, = ax.plot([min(il5),max(il5)], np.repeat(0.8,2), label='IL RX5', marker='o', color='tab:cyan')
    
    # FS/N +- HD
    fs2_m_hd2_line, = ax.plot([min(fs2_m_hd2),max(fs2_m_hd2)], np.repeat(2.5,2), label='FS/2-HD2', marker='o', color='lightsalmon')
    fs2_m_hd3_line, = ax.plot([min(fs2_m_hd3),max(fs2_m_hd3)], np.repeat(3.5,2), label='FS/2-HD3', marker='o', color='bisque')
    fs4_m_hd2_line, = ax.plot([min(fs4_m_hd2),max(fs4_m_hd2)], np.repeat(2.5,2), label='FS/4-HD2', marker='o', color='gold')    
    fs4_p_hd2_line, = ax.plot([min(fs4_p_hd2),max(fs4_p_hd2)], np.repeat(2.5,2), label='FS/4+HD2', marker='o', color='sienna')
    fs4_m_hd3_line, = ax.plot([min(fs4_m_hd3),max(fs4_m_hd3)], np.repeat(3.5,2), label='FS/4-HD3', marker='o', color='olivedrab')
    fs4_p_hd3_line, = ax.plot([min(fs4_p_hd3),max(fs4_p_hd3)], np.repeat(3.5,2), label='FS/4+HD3', marker='o', color='greenyellow')
    fs8_m_hd2_line, = ax.plot([min(fs8_m_hd2),max(fs8_m_hd2)], np.repeat(2.5,2), label='FS/8-HD2', marker='o', color='steelblue')
    fs8_p_hd2_line, = ax.plot([min(fs8_p_hd2),max(fs8_p_hd2)], np.repeat(2.5,2), label='FS/8+HD2', marker='o', color='palegreen')
    fs8_m_hd3_line, = ax.plot([min(fs8_m_hd3),max(fs8_m_hd3)], np.repeat(3.5,2), label='FS/8-HD3', marker='o', color='slategrey')
    fs8_p_hd3_line, = ax.plot([min(fs8_p_hd3),max(fs8_p_hd3)], np.repeat(3.5,2), label='FS/8+HD3', marker='o', color='powderblue')
    
    # PLL
    pll_up_line, = ax.plot([min(clk_mix_prod_up),max(clk_mix_prod_up)], np.repeat(4.5,2), label='PLL Mixing', marker='o', color='palevioletred')
    pll_dwn_line, = ax.plot([min(clk_mix_prod_down),max(clk_mix_prod_down)], np.repeat(4.5,2), label='PLL Mixing', marker='o', color='plum')
    
    ax.legend(bbox_to_anchor=(1.05,1),fontsize=9, borderaxespad=0., loc=2)
    fig.tight_layout()
    
    return (fig, ax, 
            [rx_line_a, rx_line_b, rx_line_c], 
            [nyq_line_a, nyq_line_b], 
            [hd2_line, hd3_line, hd4_line, hd5_line], 
            [il1_line, il2_line, il3_line, il4_line, il5_line], 
            [fs2_m_hd2_line, fs2_m_hd3_line, fs4_m_hd2_line, fs4_p_hd2_line, fs4_m_hd3_line, fs4_p_hd3_line, fs8_m_hd2_line, fs8_p_hd2_line, fs8_m_hd3_line, fs8_p_hd3_line],
            [pll_up_line, pll_dwn_line]
           )

def get_calibration_mode(fs_rf, fc):
    if ((fs_rf/2*0.7 < fc) and (fc < fs_rf/2*1.3)) or ((fs_rf/2*3-0.3*fs_rf/2 < fc) and (fc < fs_rf/2*3+0.3*fs_rf/2)) or ((fs_rf/2*5-0.3*fs_rf/2 < fc) and (fc < fs_rf/2*5+0.3*fs_rf/2)) or ((fs_rf/2*7-0.3*fs_rf/2 < fc) and (fc < fs_rf/2*7+0.3*rf_fs/2)) or ((fs_rf/2*9+0.3*fs_rf/2 < fc) and (fc < fs_rf/2*9+0.3*fs_rf/2)):
        return "Calibration Mode: Mode 1"
    else:
        return "Calibration Mode: Mode 2" 
    
def update_plot(change):
    with output:
        f, signal_f, interf_f, hd, il, fs_hd, clk_mix_prod_up, clk_mix_prod_down = get_signals(fs_slider.value, fc_slider.value, bw_slider.value, pll_slider.value, int(ilf_widget.value))
        hd2, hd3, hd4, hd5 = hd
        il1, il2, il3, il4, il5, il6, il7 = il
        fs2_m_hd2, fs2_m_hd3, fs4_m_hd2, fs4_p_hd2, fs4_m_hd3, fs4_p_hd3, fs8_m_hd2, fs8_p_hd2, fs8_m_hd3, fs8_p_hd3 = fs_hd
        
#         updated_values = [hd2, hd3, hd4, hd5, il1, il2, il3, il4, il5, il6, il7, fs2_m_hd2, fs2_m_hd3, fs4_m_hd2, fs4_p_hd2, fs4_m_hd3, fs4_p_hd3, fs8_m_hd2, fs8_p_hd2, fs8_m_hd3, fs8_p_hd3]
        
        ax.set_xlim([0, fs_slider.value/2 + fs_slider.value*0.01])
        # Rx Band
        rx_line[0].set_xdata([min(signal_f), max(signal_f)])
        rx_line[1].set_xdata(min(signal_f))
        rx_line[2].set_xdata(max(signal_f))
        
        # Nyquist Band
        nyq_line[0].set_xdata(fs_slider.value/2)
        nyq_line[1].set_xdata(fs_slider.value/2)
        
        # Harmonics
        hd_line[0].set_xdata([min(hd2), max(hd2)])
        hd_line[1].set_xdata([min(hd3), max(hd3)])
        hd_line[2].set_xdata([min(hd4), max(hd4)])
        hd_line[3].set_xdata([min(hd5), max(hd5)])
        
        # Interleaving Spur
        if il1 != 0: il_line[0].set_xdata([min(il1), max(il1)]) 
        if il2 != 0: il_line[1].set_xdata([min(il2), max(il2)])
        if il3 != 0: il_line[2].set_xdata([min(il3), max(il3)])
        if il4 != 0: il_line[3].set_xdata([min(il4), max(il4)])
        if il5 != 0: il_line[4].set_xdata([min(il5), max(il5)])
            
        # FS/N +- HD
        if fs2_m_hd2 != 0: fs_hd_line[0].set_xdata([min(fs2_m_hd2),max(fs2_m_hd2)])
        if fs2_m_hd3 != 0: fs_hd_line[1].set_xdata([min(fs2_m_hd3),max(fs2_m_hd3)])
        if fs4_m_hd2 != 0: fs_hd_line[2].set_xdata([min(fs4_m_hd2),max(fs4_m_hd2)])
        if fs4_p_hd2 != 0: fs_hd_line[3].set_xdata([min(fs4_p_hd2),max(fs4_p_hd2)])
        if fs4_m_hd3 != 0: fs_hd_line[4].set_xdata([min(fs4_m_hd3),max(fs4_m_hd3)])
        if fs4_p_hd3 != 0: fs_hd_line[5].set_xdata([min(fs4_p_hd3),max(fs4_p_hd3)])
        if fs8_m_hd2 != 0: fs_hd_line[6].set_xdata([min(fs8_m_hd2),max(fs8_m_hd2)])
        if fs8_p_hd2 != 0: fs_hd_line[7].set_xdata([min(fs8_p_hd2),max(fs8_p_hd2)])
        if fs8_m_hd3 != 0: fs_hd_line[8].set_xdata([min(fs8_m_hd3),max(fs8_m_hd3)])
        if fs8_p_hd3 != 0: fs_hd_line[9].set_xdata([min(fs8_p_hd3),max(fs8_p_hd3)])
            
        # PLL
        pll_line[0].set_xdata([min(clk_mix_prod_up),max(clk_mix_prod_up)])
        pll_line[1].set_xdata([min(clk_mix_prod_down),max(clk_mix_prod_down)])
        
        mixmode.value = get_calibration_mode(fs_slider.value, fc_slider.value)
        
        # Intersections
        plot_buttons[0].button_style = 'success' if intersection(hd2, signal_f) == False else 'danger'
        plot_buttons[1].button_style = 'success' if intersection(hd3, signal_f) == False else 'danger'
        plot_buttons[2].button_style = 'success' if intersection(hd4, signal_f) == False else 'danger'
        plot_buttons[3].button_style = 'success' if intersection(hd5, signal_f) == False else 'danger'
        
        plot_buttons[4].button_style = 'success' if intersection(il1, signal_f) == False else 'warning'
        plot_buttons[5].button_style = 'success' if intersection(il2, signal_f) == False else 'warning'
        plot_buttons[6].button_style = 'success' if intersection(il3, signal_f) == False else 'warning'
        plot_buttons[7].button_style = 'success' if intersection(il4, signal_f) == False else 'warning'
        plot_buttons[8].button_style = 'success' if intersection(il5, signal_f) == False else 'warning'
        
        plot_buttons[9].button_style = 'success' if intersection(fs2_m_hd2, signal_f) == False else 'warning'
        plot_buttons[10].button_style = 'success' if intersection(fs2_m_hd3, signal_f) == False else 'warning'
        plot_buttons[11].button_style = 'success' if intersection(fs4_m_hd2, signal_f) == False else 'warning'
        plot_buttons[12].button_style = 'success' if intersection(fs4_p_hd2, signal_f) == False else 'warning'
        plot_buttons[13].button_style = 'success' if intersection(fs4_m_hd3, signal_f) == False else 'warning'
        plot_buttons[14].button_style = 'success' if intersection(fs4_p_hd3, signal_f) == False else 'warning'
        plot_buttons[15].button_style = 'success' if intersection(fs8_m_hd2, signal_f) == False else 'warning'
        plot_buttons[16].button_style = 'success' if intersection(fs8_p_hd2, signal_f) == False else 'warning'
        plot_buttons[17].button_style = 'success' if intersection(fs8_m_hd3, signal_f) == False else 'warning'
        plot_buttons[18].button_style = 'success' if intersection(fs8_p_hd3, signal_f) == False else 'warning'
        
        if intersection(clk_mix_prod_up, signal_f) == False or intersection(clk_mix_prod_down, signal_f) == False:
            plot_buttons[19].button_style = 'success' 
        else:
            plot_buttons[19].button_style = 'warning'                                                  
        
        fig.canvas.draw()
        
def toggle_lines(change):
    with output:
        hd_line[0].set_visible(plot_buttons[0].value)
        hd_line[1].set_visible(plot_buttons[1].value)
        hd_line[2].set_visible(plot_buttons[2].value)
        hd_line[3].set_visible(plot_buttons[3].value)
        
        il_line[0].set_visible(plot_buttons[4].value)
        il_line[1].set_visible(plot_buttons[5].value)
        il_line[2].set_visible(plot_buttons[6].value)
        il_line[3].set_visible(plot_buttons[7].value)
        il_line[4].set_visible(plot_buttons[8].value)
        
        fs_hd_line[0].set_visible(plot_buttons[9].value)
        fs_hd_line[1].set_visible(plot_buttons[10].value)
        fs_hd_line[2].set_visible(plot_buttons[11].value)
        fs_hd_line[3].set_visible(plot_buttons[12].value)
        fs_hd_line[4].set_visible(plot_buttons[13].value)
        fs_hd_line[5].set_visible(plot_buttons[14].value)
        fs_hd_line[6].set_visible(plot_buttons[15].value)
        fs_hd_line[7].set_visible(plot_buttons[16].value)
        fs_hd_line[8].set_visible(plot_buttons[17].value)
        fs_hd_line[9].set_visible(plot_buttons[18].value)
        
        pll_line[0].set_visible(plot_buttons[19].value)
        pll_line[1].set_visible(plot_buttons[19].value)
        
def intersection(a,signal_f):
    if a:
        if ((min(a) < max(signal_f)) and (max(a) < min(signal_f))) or ((min(a) > max(signal_f)) and (max(a) > min(signal_f))):
            return False
        else:
            return True
    else:
        return False
        

In [137]:
# initial setup
fs_rf = 4092
fc = 1800
fs_bw = 20
pll_ref = 250
il_factor = 8
f, signal_f, interf_f, hd, il, fs_hd, clk_mix_prod_up, clk_mix_prod_down = get_signals(fs_rf, fc, fs_bw, pll_ref, il_factor)

In [138]:
fig, ax, rx_line, nyq_line, hd_line, il_line, fs_hd_line, pll_line = make_plot(signal_f, fs_rf, hd, il, fs_hd)

entry_layout = widgets.Layout(width='80px')
slider_layout = widgets.Layout(width='250px')
params_grid_layout = widgets.Layout(width='auto',
                                   grid_template_columns='260px 90px 90px 90px 90px 90px 90px 90px',
                                   grid_template_rows='auto auto auto auto auto auto',
                                   grid_gap='5px')

# Params Widgets
fs_slider = widgets.FloatSlider(value=4096.0, min=1000.0, max=4096.0, step=0.01, description='Sample Rate', continuous_update=True, readout=False, layout=slider_layout)
fs_entry  = widgets.BoundedFloatText(value=4096.0, min=1000.0, max=4096.0, step=1.0, continuous_update=False, layout=entry_layout)

fc_slider = widgets.FloatSlider(value=1800.0, min=0.0, max=4096.0, step=0.01, description='Signal Centre', continuous_update=True, readout=False, layout=slider_layout)
fc_entry  = widgets.BoundedFloatText(value=1800.0, min=0.0, max=4096.0, step=1.0, continuous_update=False, layout=entry_layout)

bw_slider = widgets.FloatSlider(value=20.0, min=0.0, max=500.0, step=0.01, description='Signal BW', continuous_update=True, readout=False, layout=slider_layout)
bw_entry = widgets.BoundedFloatText(value=20.0, min=0.0, max=500.0, step=1, continuous_update=False, layout=entry_layout)

pll_slider = widgets.FloatSlider(value=250.0, min=110.0, max=614.0, step=0.01, description='PLL Ref', continuous_update=True, readout=False, layout=slider_layout)
pll_entry = widgets.BoundedFloatText(value=250.0, min=110.0, max=614.0, step=1, continuous_update=False, layout=entry_layout)

ilf_widget = widgets.Dropdown(options=["4","8"],value="8",description="IL Factor", layout=widgets.Layout(width='150px'))

l_fs = widgets.jslink((fs_slider, 'value'), (fs_entry, 'value'))
l_fc = widgets.jslink((fc_slider, 'value'), (fc_entry, 'value'))
l_bw = widgets.jslink((bw_slider, 'value'), (bw_entry, 'value'))
l_pll = widgets.jslink((pll_slider, 'value'), (pll_entry, 'value'))

blank_widget = widgets.Label("", format=widgets.Layout(border='solid'))

mixmode = widgets.Label("Calibration Mode: ")

params_units = [widgets.Label(item) for item in ["Msps", "MHz", "MHz", "MHz"]]

# Plot Items Toggle
plot_items = ["HD2", "HD3", "HD4", "HD5", "IL1", "IL2", "IL3", "IL4", "IL5", "FS/2-HD2", "FS/2-HD3", "FS/4-HD2", "FS/4+HD2", "FS/4-HD3", "FS/4+HD3", "FS/8-HD2", "FS/8+HD2", "FS/8-HD3", "FS/8+HD3", "PLL Mixing"]
plot_buttons = [widgets.ToggleButton(description=i, value=True, layout=widgets.Layout(width='90px')) for i in plot_items]
# set up grid
grid_order = [fs_slider, fs_entry, params_units[0], plot_buttons[0], plot_buttons[4], plot_buttons[8], plot_buttons[12], plot_buttons[16],
          fc_slider, fc_entry, params_units[1], plot_buttons[1], plot_buttons[5], plot_buttons[9], plot_buttons[13], plot_buttons[17],
          bw_slider, bw_entry, params_units[2], plot_buttons[2], plot_buttons[6], plot_buttons[10], plot_buttons[14], plot_buttons[18],
          pll_slider, pll_entry, params_units[3], plot_buttons[3], plot_buttons[7], plot_buttons[11], plot_buttons[15], plot_buttons[19],
          ilf_widget, blank_widget, blank_widget, blank_widget, blank_widget, blank_widget, blank_widget, blank_widget,
              mixmode
             ]

grid = widgets.GridBox(children=grid_order, layout=params_grid_layout)

# update plot
fs_slider.observe(update_plot, 'value')
fs_entry.observe(update_plot, 'value')
fc_slider.observe(update_plot, 'value')
bw_slider.observe(update_plot, 'value')
pll_slider.observe(update_plot, 'value')
ilf_widget.observe(update_plot, 'value')

[button.observe(toggle_lines, 'value') for button in plot_buttons]

output = widgets.Output()
# display(fs_slider, fc_slider, bw_slider, pll_slider, ilf_widget, output)
display(grid)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

GridBox(children=(FloatSlider(value=4096.0, description='Sample Rate', layout=Layout(width='250px'), max=4096.…